In [1]:
import pandas as pd
import numpy as np

# Loading dataset

df_awards = pd.read_csv('Player Award Shares.csv')
df_advanced = pd.read_csv('Advanced.csv')
df_per_game = pd.read_csv('Player Per Game.csv')
df_team_summary = pd.read_csv('Team Summaries.csv')
df_career_info = pd.read_csv('Player Career Info.csv')
df_draft = pd.read_csv('Draft Pick History.csv')

# Check the head of each table
print("--- Player Award Shares.csv loaded: ---")
print(df_awards.head())

print("\n--- Advanced.csv loaded: ---")
print(df_advanced.head())

print("\n--- Player Per Game.csv loaded: ---")
print(df_per_game.head())

print("\n--- Team Summaries.csv loaded: ---")
print(df_team_summary.head())

print("\n--- Player Career Info.csv loaded: ---")
print(df_career_info.head())

print("\n--- Draft Pick History.csv loaded: ---")
print(df_draft.head())

--- Player Award Shares.csv loaded: ---
   season           award                   player  player_id  age  first  \
0    2024  nba clutch_poy            Stephen Curry  curryst01   35   45.0   
1    2024  nba clutch_poy            DeMar DeRozan  derozde01   34   34.0   
2    2024  nba clutch_poy  Shai Gilgeous-Alexander  gilgesh01   25   11.0   
3    2024  nba clutch_poy             Nikola Jokić  jokicni01   28    4.0   
4    2024  nba clutch_poy            Jalen Brunson  brunsja01   27    2.0   

   pts_won  pts_max  share  winner  
0    298.0    495.0  0.602    True  
1    272.0    495.0  0.549   False  
2    160.0    495.0  0.323   False  
3     85.0    495.0  0.172   False  
4     26.0    495.0  0.053   False  

--- Advanced.csv loaded: ---
   season   lg            player  player_id   age team pos   g    gs      mp  \
0    2025  NBA  Precious Achiuwa  achiupr01  25.0  NYK   C  57  10.0  1170.0   
1    2025  NBA      Steven Adams  adamsst01  31.0  HOU   C  58   3.0   794.0   
2    

In [2]:
# Step 2: Filter the seasonal DataFrames to the 2010-2024 range

print("Filtering seasonal dataframes from 2010 to 2024...")

# We use two conditions: (season >= 2010) AND (season <= 2024)
# We add .copy() to avoid a "SettingWithCopyWarning" in pandas

df_awards_2010_2024 = df_awards[
    (df_awards['season'] >= 2010) & 
    (df_awards['season'] <= 2024)
].copy()

df_advanced_2010_2024 = df_advanced[
    (df_advanced['season'] >= 2010) & 
    (df_advanced['season'] <= 2024)
].copy()

df_per_game_2010_2024 = df_per_game[
    (df_per_game['season'] >= 2010) & 
    (df_per_game['season'] <= 2024)
].copy()

df_team_summary_2010_2024 = df_team_summary[
    (df_team_summary['season'] >= 2010) & 
    (df_team_summary['season'] <= 2024)
].copy()

# --- The "lookup" tables (df_career_info, df_draft) are NOT filtered ---
# We need their full history to look up players who were drafted before 2010
# but were MVP candidates between 2010-2024.


# --- Verify the filtering ---
print("\n--- df_awards (filtered) ---")
print(f"Shape: {df_awards_2010_2024.shape}")
print("Head (oldest):")
print(df_awards_2010_2024.head())
print("\nTail (newest):")
print(df_awards_2010_2024.tail())

print("\n--- df_team_summary (filtered) ---")
print(f"Shape: {df_team_summary_2010_2024.shape}")
print("Season min:", df_team_summary_2010_2024['season'].min())
print("Season max:", df_team_summary_2010_2024['season'].max())

Filtering seasonal dataframes from 2010 to 2024...

--- df_awards (filtered) ---
Shape: (1159, 10)
Head (oldest):
   season           award                   player  player_id  age  first  \
0    2024  nba clutch_poy            Stephen Curry  curryst01   35   45.0   
1    2024  nba clutch_poy            DeMar DeRozan  derozde01   34   34.0   
2    2024  nba clutch_poy  Shai Gilgeous-Alexander  gilgesh01   25   11.0   
3    2024  nba clutch_poy             Nikola Jokić  jokicni01   28    4.0   
4    2024  nba clutch_poy            Jalen Brunson  brunsja01   27    2.0   

   pts_won  pts_max  share  winner  
0    298.0    495.0  0.602    True  
1    272.0    495.0  0.549   False  
2    160.0    495.0  0.323   False  
3     85.0    495.0  0.172   False  
4     26.0    495.0  0.053   False  

Tail (newest):
      season     award           player  player_id  age  first  pts_won  \
1154    2010  nba smoy  Marcus Thornton  thornma01   22    0.0      3.0   
1155    2010  nba smoy     Jared Du

In [3]:
# --- Step 3: Clean and Select Relevant Columns ---

# --- Clean the DataFrames filtered in the previous step ---

# 1. Clean MVP Awards Data
# We only want the "nba mvp" award and the key voting info
df_awards_clean = df_awards_2010_2024[
    df_awards_2010_2024['award'] == 'nba mvp'
].copy()
df_awards_clean = df_awards_clean[['season', 'player_id', 'share', 'winner']]

print("--- Cleaned Awards (df_awards_clean) ---")
print(df_awards_clean.head())


# 2. Clean Team Summary Data
# We only need the team's wins (w)
df_team_wins_clean = df_team_summary_2010_2024[['season', 'abbreviation', 'w']].copy()
# Rename 'abbreviation' to 'team' so it can be merged with player data
df_team_wins_clean.rename(columns={'abbreviation': 'team'}, inplace=True)

print("\n--- Cleaned Team Wins (df_team_wins_clean) ---")
print(df_team_wins_clean.head())


# 3. Clean Advanced Player Stats
# We need 'g' (games) to find the player's primary team if traded
df_advanced_clean = df_advanced_2010_2024[
    ['season', 'player_id', 'team', 'g', 'per', 'ws', 'vorp']
].copy()

print("\n--- Cleaned Advanced Stats (df_advanced_clean) ---")
print(df_advanced_clean.head())


# 4. Clean Per Game Player Stats
# We also need 'g' here, plus the basic counting stats
df_per_game_clean = df_per_game_2010_2024[
    ['season', 'player_id', 'team', 'g', 'pts_per_game', 'trb_per_game', 'ast_per_game']
].copy()

print("\n--- Cleaned Per Game Stats (df_per_game_clean) ---")
print(df_per_game_clean.head())


# --- Clean the original, UNFILTERED "lookup" DataFrames ---

# 5. Clean Career Info
# We only need college info. We drop duplicates to keep one row per player.
df_career_clean = df_career_info[['player_id', 'colleges']].drop_duplicates().copy()

print("\n--- Cleaned Career Info (df_career_clean) ---")
print(df_career_clean.head())


# 6. Clean Draft History
# We only need draft pick. We drop duplicates.
df_draft_clean = df_draft[['player_id', 'overall_pick']].drop_duplicates().copy()

print("\n--- Cleaned Draft History (df_draft_clean) ---")
print(df_draft_clean.head())

--- Cleaned Awards (df_awards_clean) ---
    season  player_id  share  winner
42    2024  jokicni01  0.935    True
43    2024  gilgesh01  0.646   False
44    2024  doncilu01  0.572   False
45    2024  antetgi01  0.194   False
46    2024  brunsja01  0.143   False

--- Cleaned Team Wins (df_team_wins_clean) ---
    season team     w
31    2024  ATL  36.0
32    2024  BOS  64.0
33    2024  BRK  32.0
34    2024  CHI  39.0
35    2024  CHO  21.0

--- Cleaned Advanced Stats (df_advanced_clean) ---
     season  player_id team   g   per   ws  vorp
735    2024  achiupr01  2TM  74  14.6  3.4   0.2
736    2024  achiupr01  TOR  25  15.0  0.4   0.0
737    2024  achiupr01  NYK  49  14.5  3.0   0.2
738    2024  adebaba01  MIA  71  19.8  7.2   2.7
739    2024  agbajoc01  2TM  78   7.7  0.1  -1.0

--- Cleaned Per Game Stats (df_per_game_clean) ---
     season  player_id team   g  pts_per_game  trb_per_game  ast_per_game
735    2024  achiupr01  2TM  74           7.6           6.6           1.3
736    2024

In [4]:
# --- Step 3.5: Check for Missing Values (NaN) Before Merging ---

print("--- 1. Checking df_awards_clean ---")
# We need: season, player_id (Keys)
# We need: share, winner (Values)
print(df_awards_clean.isnull().sum())


print("\n--- 2. Checking df_team_wins_clean ---")
# We need: season, team (Keys)
# We need: w (Value)
print(df_team_wins_clean.isnull().sum())


print("\n--- 3. Checking df_advanced_clean ---")
# We need: season, player_id, team, g (Keys)
# We need: per, ws, vorp (Values)
# Note: 'per', 'ws', 'vorp' might have NaNs if a player played 0 minutes
# but was still in the table. Our 2010-2024 filter should remove most of this.
print(df_advanced_clean.isnull().sum())


print("\n--- 4. Checking df_per_game_clean ---")
# We need: season, player_id, team, g (Keys)
# We need: pts_per_game, trb_per_game, ast_per_game (Values)
print(df_per_game_clean.isnull().sum())


print("\n--- 5. Checking df_career_clean (Lookup Table) ---")
# We need: player_id (Key)
# We need: colleges (Value)
# 'colleges' *will* have NaNs (for international/HS players), which is OK!
print(df_career_clean.isnull().sum())


print("\n--- 6. Checking df_draft_clean (Lookup Table) ---")
# We need: player_id (Key)
# We need: overall_pick (Value)
# 'overall_pick' *will* have NaNs (for undrafted players), which is OK!
print(df_draft_clean.isnull().sum())

--- 1. Checking df_awards_clean ---
season       0
player_id    0
share        0
winner       0
dtype: int64

--- 2. Checking df_team_wins_clean ---
season     0
team      15
w         15
dtype: int64

--- 3. Checking df_advanced_clean ---
season       0
player_id    0
team         0
g            0
per          2
ws           0
vorp         0
dtype: int64

--- 4. Checking df_per_game_clean ---
season          0
player_id       0
team            0
g               0
pts_per_game    0
trb_per_game    0
ast_per_game    0
dtype: int64

--- 5. Checking df_career_clean (Lookup Table) ---
player_id      0
colleges     396
dtype: int64

--- 6. Checking df_draft_clean (Lookup Table) ---
player_id         0
overall_pick    423
dtype: int64


In [6]:
# --- Step 3.6: Check for UNEXPECTED Duplicates Before Merging ---

print("--- Checking for UNEXPECTED Duplicates Before Merging ---")

# 1. df_awards_clean
# Key = (season, player_id) - A player should only have one MVP entry per season.
key_awards = ['season', 'player_id']
duplicates_awards = df_awards_clean.duplicated(subset=key_awards).sum()
print(f"1. df_awards_clean: {duplicates_awards} duplicates found on (season, player_id)")

# 2. df_team_wins_clean
# Key = (season, team) - A team should only have one win record per season.
key_team = ['season', 'team']
duplicates_team = df_team_wins_clean.duplicated(subset=key_team).sum()
print(f"\n2. df_team_wins_clean: {duplicates_team} duplicates found on (season, team)")

# 3. df_advanced_clean
# Key = (season, player_id, team) - A player should only have one row *per team* per season.
key_player_team = ['season', 'player_id', 'team']
duplicates_advanced = df_advanced_clean.duplicated(subset=key_player_team).sum()
print(f"\n3. df_advanced_clean: {duplicates_advanced} duplicates found on (season, player_id, team)")

# 4. df_per_game_clean
# Key = (season, player_id, team) - Same as above.
duplicates_per_game = df_per_game_clean.duplicated(subset=key_player_team).sum()
print(f"\n4. df_per_game_clean: {duplicates_per_game} duplicates found on (season, player_id, team)")

# 5. df_career_clean (Lookup Table)
# Key = (player_id) - A player should only have one career entry.
# Note: We already ran .drop_duplicates() in Step 3, so this should be 0.
key_career = ['player_id']
duplicates_career = df_career_clean.duplicated(subset=key_career).sum()
print(f"\n5. df_career_clean: {duplicates_career} duplicates found on (player_id)")

# 6. df_draft_clean (Lookup Table)
# Key = (player_id) - A player can only be drafted once.
# Note: We also ran .drop_duplicates() in Step 3, this should also be 0.
key_draft = ['player_id']
duplicates_draft = df_draft_clean.duplicated(subset=key_draft).sum()
print(f"\n6. df_draft_clean: {duplicates_draft} duplicates found on (player_id)")

--- Checking for UNEXPECTED Duplicates Before Merging ---
1. df_awards_clean: 0 duplicates found on (season, player_id)

2. df_team_wins_clean: 0 duplicates found on (season, team)

3. df_advanced_clean: 0 duplicates found on (season, player_id, team)

4. df_per_game_clean: 0 duplicates found on (season, player_id, team)

5. df_career_clean: 0 duplicates found on (player_id)

6. df_draft_clean: 152 duplicates found on (player_id)


In [8]:
# --- Step 3.7: Fix Draft Duplicates and Re-Verify ---

df_draft = pd.read_csv('Draft Pick History.csv')

print("--- Fixing df_draft_clean ---")

# --- THIS IS THE FIX ---
# We are REPLACING the old df_draft_clean.
# We now tell pandas to drop duplicates based ONLY on 'player_id',
# and to keep the 'first' entry it finds.
df_draft_clean = df_draft[['player_id', 'overall_pick']].drop_duplicates(
    subset=['player_id'],
    keep='first'
).copy()

print("New df_draft_clean has been created.")


# --- RE-RUNNING THE CHECK ---
print("\n--- Re-Checking for Duplicates ---")
key_draft = ['player_id']
duplicates_draft = df_draft_clean.duplicated(subset=key_draft).sum()

print(f"6. df_draft_clean: {duplicates_draft} duplicates found on (player_id)")

if duplicates_draft == 0:
    print("\nSUCCESS! The 152 duplicates have been fixed.")
    print("We are now 100% ready for Step 4.")
else:
    print("\nError: Duplicates still exist. Please check the code.")

--- Fixing df_draft_clean ---
New df_draft_clean has been created.

--- Re-Checking for Duplicates ---
6. df_draft_clean: 0 duplicates found on (player_id)

SUCCESS! The 152 duplicates have been fixed.
We are now 100% ready for Step 4.


In [9]:
# --- Step 3.8: Fix Critical Missing Values and Re-Verify ---

print("--- Fixing Missing Values ---")

# 1. Fix df_team_wins_clean (15 missing 'team' and 'w')
# We will drop any row that doesn't have a 'team' OR doesn't have a 'w'
original_count_team = len(df_team_wins_clean)
df_team_wins_clean = df_team_wins_clean.dropna(subset=['team', 'w']).copy()
new_count_team = len(df_team_wins_clean)
print(f"Fixed df_team_wins_clean: Removed {original_count_team - new_count_team} rows with missing 'team' or 'w'.")

# 2. Fix df_advanced_clean (2 missing 'per')
# We will drop any row that doesn't have a 'per' value
original_count_adv = len(df_advanced_clean)
df_advanced_clean = df_advanced_clean.dropna(subset=['per']).copy()
new_count_adv = len(df_advanced_clean)
print(f"Fixed df_advanced_clean: Removed {original_count_adv - new_count_adv} rows with missing 'per'.")


# --- RE-RUNNING THE CHECK ---
print("\n--- Re-Checking Missing Values After Fix ---")

# Check 2
print("\n--- 2. Checking df_team_wins_clean ---")
print(df_team_wins_clean.isnull().sum())

# Check 3
print("\n--- 3. Checking df_advanced_clean ---")
print(df_advanced_clean.isnull().sum())

print("\n--- STATUS ---")
print("If all counts above are now '0', you are ready for Step 4 (Merging).")

--- Fixing Missing Values ---
Fixed df_team_wins_clean: Removed 15 rows with missing 'team' or 'w'.
Fixed df_advanced_clean: Removed 2 rows with missing 'per'.

--- Re-Checking Missing Values After Fix ---

--- 2. Checking df_team_wins_clean ---
season    0
team      0
w         0
dtype: int64

--- 3. Checking df_advanced_clean ---
season       0
player_id    0
team         0
g            0
per          0
ws           0
vorp         0
dtype: int64

--- STATUS ---
If all counts above are now '0', you are ready for Step 4 (Merging).


In [11]:
# --- Step 3.9: Check for Outliers using .describe() ---

print("--- 1. Checking 'df_advanced_clean' Stats ---")
# We select the key numerical columns to check
print(df_advanced_clean[['g', 'per', 'ws', 'vorp']].describe())

print("\n--- 2. Checking 'df_per_game_clean' Stats ---")
print(df_per_game_clean[['g', 'pts_per_game', 'trb_per_game', 'ast_per_game']].describe())

print("\n--- 3. Checking 'df_team_wins_clean' Stats ---")
print(df_team_wins_clean[['w']].describe())

print("\n--- 4. Checking 'df_awards_clean' Stats ---")
print(df_awards_clean[['share']].describe())

print("\n--- 5. Checking 'df_draft_clean' Stats ---")
# We filter out 0 (our 'Undrafted' placeholder) to get a real sense of draft picks
print(df_draft_clean[df_draft_clean['overall_pick'] > 0][['overall_pick']].describe())

--- 1. Checking 'df_advanced_clean' Stats ---
                 g          per           ws         vorp
count  9714.000000  9714.000000  9714.000000  9714.000000
mean     43.517706    12.586267     2.037894     0.470311
std      25.662870     6.668645     2.557628     1.131908
min       1.000000   -45.200000    -2.100000    -2.000000
25%      20.000000     9.500000     0.200000    -0.100000
50%      46.000000    12.600000     1.100000     0.100000
75%      67.000000    15.900000     3.100000     0.700000
max      84.000000   133.800000    19.300000    10.600000

--- 2. Checking 'df_per_game_clean' Stats ---
                 g  pts_per_game  trb_per_game  ast_per_game
count  9716.000000   9716.000000   9716.000000   9716.000000
mean     43.509057      8.048281      3.411301      1.807112
std      25.667309      5.913770      2.387056      1.763727
min       1.000000      0.000000      0.000000      0.000000
25%      20.000000      3.600000      1.700000      0.600000
50%      46.000000 

In [13]:
# --- Step 3.10: Final Data Type Validation (Before Merging) ---

print("--- 1. df_awards_clean Types ---")
print(df_awards_clean.dtypes)

print("\n--- 2. df_team_wins_clean Types ---")
print(df_team_wins_clean.dtypes)

print("\n--- 3. df_advanced_clean Types ---")
print(df_advanced_clean.dtypes)

print("\n--- 4. df_per_game_clean Types ---")
print(df_per_game_clean.dtypes)

print("\n--- 5. df_career_clean Types ---")
print(df_career_clean.dtypes)

print("\n--- 6. df_draft_clean Types ---")
print(df_draft_clean.dtypes)

--- 1. df_awards_clean Types ---
season         int64
player_id     object
share        float64
winner          bool
dtype: object

--- 2. df_team_wins_clean Types ---
season      int64
team       object
w         float64
dtype: object

--- 3. df_advanced_clean Types ---
season         int64
player_id     object
team          object
g              int64
per          float64
ws           float64
vorp         float64
dtype: object

--- 4. df_per_game_clean Types ---
season            int64
player_id        object
team             object
g                 int64
pts_per_game    float64
trb_per_game    float64
ast_per_game    float64
dtype: object

--- 5. df_career_clean Types ---
player_id    object
colleges     object
dtype: object

--- 6. df_draft_clean Types ---
player_id        object
overall_pick    float64
dtype: object


In [14]:
# --- Step 4: Merge Seasonal Dataframes & Handle Traded Players ---

print("--- Starting Step 4: Merging Seasonal Data ---")

# 1. Merge player's advanced and per_game stats
# We use an 'inner' merge to ensure we only have players present in both tables
# We need 'g' from both, so we'll drop one after checking
df_player_stats = pd.merge(
    df_advanced_clean,
    df_per_game_clean,
    on=['season', 'player_id', 'team'],
    how='inner',
    suffixes=('_adv', '_pg') # Suffixes for duplicate 'g' column
)

# Now we can drop the duplicate 'g' column
# and keep the other (e.g., 'g_adv') and rename it
df_player_stats = df_player_stats.drop(columns=['g_pg'])
df_player_stats = df_player_stats.rename(columns={'g_adv': 'g'})
print(f"Merged player stats. Shape: {df_player_stats.shape}")

# 2. Merge player stats with team wins
# Using 'how='inner'' will automatically drop all 'team' == 'TOT' rows,
# because 'TOT' does not exist in 'df_team_wins_clean'.
df_stats_with_wins = pd.merge(
    df_player_stats,
    df_team_wins_clean,
    on=['season', 'team'],
    how='inner'
)
print(f"Merged with team wins. Shape: {df_stats_with_wins.shape}")

# 3. Merge with our MVP awards data
# This 'inner' merge filters our 9000+ player-seasons down to
# just the 195 rows of MVP candidates.
df_candidates = pd.merge(
    df_stats_with_wins,
    df_awards_clean,
    on=['season', 'player_id'],
    how='inner'
)
print(f"Merged with MVP awards. Shape: {df_candidates.shape}")
print(f"Total MVP candidate rows found: {len(df_candidates)}")


# 4. Handle Traded Players
# At this point, a traded player might have multiple rows.
# We will sort by 'g' (games played) and keep only the row
# for the team they played the MOST games for.

df_candidates_sorted = df_candidates.sort_values(by='g', ascending=False)
df_mvp_candidates_final = df_candidates_sorted.drop_duplicates(
    subset=['season', 'player_id'],
    keep='first'
)

print(f"\nHandled traded players. Final seasonal shape: {df_mvp_candidates_final.shape}")
print("--- Step 4 Complete ---")
print("This new DataFrame 'df_mvp_candidates_final' is ready for Step 5.")

# Display the first 5 rows
print("\n--- Preview of Step 4 Result ---")
print(df_mvp_candidates_final.head())

--- Starting Step 4: Merging Seasonal Data ---
Merged player stats. Shape: (9714, 10)
Merged with team wins. Shape: (8708, 11)
Merged with MVP awards. Shape: (198, 13)
Total MVP candidate rows found: 198

Handled traded players. Final seasonal shape: (195, 13)
--- Step 4 Complete ---
This new DataFrame 'df_mvp_candidates_final' is ready for Step 5.

--- Preview of Step 4 Result ---
     season  player_id team   g   per    ws  vorp  pts_per_game  trb_per_game  \
7      2024  sabondo01  SAC  82  23.2  12.6   6.2          19.4          13.7   
195    2010  stoudam01  PHO  82  22.6  10.7   2.2          23.1           8.9   
188    2010  howardw01  ORL  82  24.0  13.2   4.6          18.3          13.2   
118    2015   paulch01  LAC  82  26.0  16.1   7.2          19.1           4.6   
102    2016  hardeja01  HOU  82  25.3  13.3   6.8          29.0           6.1   

     ast_per_game     w  share  winner  
7             8.2  46.0  0.003   False  
195           1.0  54.0  0.004   False  
188  

In [15]:
# --- Step 5: Merge Career Info & Final Cleanup ---
# (This assumes 'df_mvp_candidates_final' exists from Step 4)
# (It also assumes 'df_career_clean' and 'df_draft_clean' exist from Step 3)

print("--- Starting Step 5: Final Merge and Cleanup ---")

# 1. Merge career (college) info
# Use 'how='left'' to keep all MVP candidates,
# even if they have no college info (e.g., international, high school)
# We assume df_mvp_candidates_final is in memory from Step 4
df_final_merged = pd.merge(
    df_mvp_candidates_final,
    df_career_clean,
    on='player_id',
    how='left'
)
print(f"Merged college info. Shape: {df_final_merged.shape}")

# 2. Merge draft info
# Use 'how='left'' to keep all MVP candidates,
# even if they were undrafted (no 'overall_pick' entry)
df_final = pd.merge(
    df_final_merged,
    df_draft_clean,
    on='player_id',
    how='left'
)
print(f"Merged draft info. Shape: {df_final.shape}")

# 3. Final Cleanup (Handling expected NaN values)
# Now we replace the 'NaN' values from our left merges
# with meaningful labels.

# Fill missing 'colleges' with 'No College Info'
# (This applies to international & high school players)
df_final['colleges'] = df_final['colleges'].fillna('No College Info')

# Fill missing 'overall_pick' with 0 (a placeholder for 'Undrafted')
# We use 0 so the column can remain a number (float).
df_final['overall_pick'] = df_final['overall_pick'].fillna(0)
print("Cleaned up NaN values in 'colleges' and 'overall_pick'.")

# --- CONGRATULATIONS! ---
print("\n--- FINAL DATASET READY FOR ANALYSIS ---")
print("Your final, clean DataFrame is 'df_final'.")
print(df_final.head())

# Optional: Check final data types and for any last-minute nulls
print("\n--- Final Data Types ---")
print(df_final.dtypes)

print("\n--- Final Missing Value Check ---")
print(df_final.isnull().sum()) # Should all be 0 now

# Save the final dataset to a new CSV file
output_filename = 'mvp_analysis_2010_2024_FINAL.csv'
df_final.to_csv(output_filename, index=False)
print(f"\nFinal dataset saved to '{output_filename}'!")

--- Starting Step 5: Final Merge and Cleanup ---
Merged college info. Shape: (195, 14)
Merged draft info. Shape: (195, 15)
Cleaned up NaN values in 'colleges' and 'overall_pick'.

--- FINAL DATASET READY FOR ANALYSIS ---
Your final, clean DataFrame is 'df_final'.
   season  player_id team   g   per    ws  vorp  pts_per_game  trb_per_game  \
0    2024  sabondo01  SAC  82  23.2  12.6   6.2          19.4          13.7   
1    2010  stoudam01  PHO  82  22.6  10.7   2.2          23.1           8.9   
2    2010  howardw01  ORL  82  24.0  13.2   4.6          18.3          13.2   
3    2015   paulch01  LAC  82  26.0  16.1   7.2          19.1           4.6   
4    2016  hardeja01  HOU  82  25.3  13.3   6.8          29.0           6.1   

   ast_per_game     w  share  winner         colleges  overall_pick  
0           8.2  46.0  0.003   False          Gonzaga          11.0  
1           1.0  54.0  0.004   False  No College Info           9.0  
2           1.8  59.0  0.389   False  No College In

In [17]:
# --- Analyze Top Colleges for MVP Candidates (2010-2024) ---

df_final = pd.read_csv('mvp_analysis_2010_2024_FINAL.csv')
print("Loaded final analysis dataset.")

# 1. Count the occurrences of each college
#    value_counts() automatically sorts in descending order
college_counts = df_final['colleges'].value_counts()

# 2. Exclude 'No College Info'
#    We use .drop() to remove this category if it exists
if 'No College Info' in college_counts.index:
    college_counts = college_counts.drop('No College Info')

# 3. Display the top N colleges (e.g., Top 10)
top_n = 10
print(f"\n--- Top {top_n} Colleges Producing MVP Candidates (2010-2024) ---")
print(college_counts.head(top_n))

# Optional: Display the count for 'No College Info' separately
no_college_count = df_final[df_final['colleges'] == 'No College Info'].shape[0]
print(f"\nNumber of MVP candidate seasons from players with 'No College Info': {no_college_count}")

# Optional: Calculate the percentage contribution
total_candidates = len(df_final)
college_counts_percent = (college_counts / total_candidates) * 100
no_college_percent = (no_college_count / total_candidates) * 100

print(f"\n--- Top {top_n} Colleges (%) ---")
print(college_counts_percent.head(top_n).round(2)) # Round to 2 decimal places
print(f"\nPercentage from 'No College Info': {no_college_percent:.2f}%")

Loaded final analysis dataset.

--- Top 10 Colleges Producing MVP Candidates (2010-2024) ---
colleges
Texas              14
Wake Forest        13
Kentucky           12
Davidson           10
UCLA               10
Arizona State       9
San Diego State     6
Kansas              5
Weber State         5
Marquette           4
Name: count, dtype: int64

Number of MVP candidate seasons from players with 'No College Info': 58

--- Top 10 Colleges (%) ---
colleges
Texas              7.18
Wake Forest        6.67
Kentucky           6.15
Davidson           5.13
UCLA               5.13
Arizona State      4.62
San Diego State    3.08
Kansas             2.56
Weber State        2.56
Marquette          2.05
Name: count, dtype: float64

Percentage from 'No College Info': 29.74%


In [18]:
# --- Analyze MVP Winner Draft Position (2010-2024) ---

df_final = pd.read_csv('mvp_analysis_2010_2024_FINAL.csv')
print("Loaded final analysis dataset.")

# 1. Filter to get only the MVP winners
mvp_winners = df_final[df_final['winner'] == True].copy()
print(f"Found {len(mvp_winners)} MVP seasons between 2010-2024.")

# 2. Separate drafted vs. undrafted winners
# Remember: overall_pick == 0 means 'Undrafted' in our cleaned data
drafted_mvps = mvp_winners[mvp_winners['overall_pick'] > 0]
undrafted_mvps = mvp_winners[mvp_winners['overall_pick'] == 0]

print(f"Number of MVP seasons from DRAFTED players: {len(drafted_mvps)}")
print(f"Number of MVP seasons from UNDRAFTED players: {len(undrafted_mvps)}") # Should be 0 in this era, but good to check

# 3. Calculate the average draft position FOR DRAFTED MVPs ONLY
if not drafted_mvps.empty:
    average_draft_pick = drafted_mvps['overall_pick'].mean()
    median_draft_pick = drafted_mvps['overall_pick'].median() # Median is often useful too
    print(f"\n--- MVP Winner Draft Position (2010-2024, Drafted Players Only) ---")
    print(f"Average Overall Pick: {average_draft_pick:.2f}")
    print(f"Median Overall Pick: {median_draft_pick}") # Median gives the middle value
else:
    print("\nNo drafted MVP winners found in this period to calculate an average.")

# Optional: List the draft picks of the winners
print("\n--- Draft Picks of MVP Winners (2010-2024) ---")
# Sort by season for clarity
mvp_winners_sorted = mvp_winners.sort_values(by='season')
# We need player names. Let's merge with career info again (just for names)
df_career_names = pd.read_csv('Player Career Info.csv')[['player_id', 'player']].drop_duplicates()
mvp_winners_with_names = pd.merge(mvp_winners_sorted, df_career_names, on='player_id', how='left')

# Display Season, Player Name, and Draft Pick (0 for undrafted)
print(mvp_winners_with_names[['season', 'player', 'overall_pick']].to_string(index=False))

Loaded final analysis dataset.
Found 15 MVP seasons between 2010-2024.
Number of MVP seasons from DRAFTED players: 15
Number of MVP seasons from UNDRAFTED players: 0

--- MVP Winner Draft Position (2010-2024, Drafted Players Only) ---
Average Overall Pick: 12.20
Median Overall Pick: 4.0

--- Draft Picks of MVP Winners (2010-2024) ---
 season                player  overall_pick
   2010          LeBron James           1.0
   2011          Derrick Rose           1.0
   2012          LeBron James           1.0
   2013          LeBron James           1.0
   2014          Kevin Durant           2.0
   2015         Stephen Curry           7.0
   2016         Stephen Curry           7.0
   2017     Russell Westbrook           4.0
   2018          James Harden           3.0
   2019 Giannis Antetokounmpo          15.0
   2020 Giannis Antetokounmpo          15.0
   2021          Nikola Jokić          41.0
   2022          Nikola Jokić          41.0
   2023           Joel Embiid           3.0
   2